# T1mapping

## Convert the nifti file into mat file

In [9]:
import os
import glob
from pathlib import Path
from tqdm import tqdm
import argparse
import scipy.io
import numpy as np
import SimpleITK as sitk

In [4]:
def nii2mat(nii_path, mat_path):
    img = sitk.ReadImage(nii_path)
    img_array = sitk.GetArrayFromImage(img)
    scipy.io.savemat(mat_path, {'img': img_array})

In [ ]:
def resize(img, new_size, interpolator):
    # img = sitk.ReadImage(img)
    dimension = img.GetDimension()

    # Physical image size corresponds to the largest physical size in the training set, or any other arbitrary size.
    reference_physical_size = np.zeros(dimension)

    reference_physical_size[:] = [(sz - 1) * spc if sz * spc > mx else mx for sz, spc, mx in
                                  zip(img.GetSize(), img.GetSpacing(), reference_physical_size)]

    # Create the reference image with a zero origin, identity direction cosine matrix and dimension
    reference_origin = np.zeros(dimension)
    reference_direction = np.identity(dimension).flatten()
    reference_size = new_size
    reference_spacing = [phys_sz / (sz - 1) for sz, phys_sz in zip(reference_size, reference_physical_size)]

    reference_image = sitk.Image(reference_size, img.GetPixelIDValue())
    reference_image.SetOrigin(reference_origin)
    reference_image.SetSpacing(reference_spacing)
    reference_image.SetDirection(reference_direction)

    # Always use the TransformContinuousIndexToPhysicalPoint to compute an indexed point's physical coordinates as
    # this takes into account size, spacing and direction cosines. For the vast majority of images the direction
    # cosines are the identity matrix, but when this isn't the case simply multiplying the central index by the
    # spacing will not yield the correct coordinates resulting in a long debugging session.
    reference_center = np.array(
        reference_image.TransformContinuousIndexToPhysicalPoint(np.array(reference_image.GetSize()) / 2.0))

    # Transform which maps from the reference_image to the current img with the translation mapping the image
    # origins to each other.
    transform = sitk.AffineTransform(dimension)
    transform.SetMatrix(img.GetDirection())
    transform.SetTranslation(np.array(img.GetOrigin()) - reference_origin)
    # Modify the transformation to align the centers of the original and reference image instead of their origins.
    centering_transform = sitk.TranslationTransform(dimension)
    img_center = np.array(img.TransformContinuousIndexToPhysicalPoint(np.array(img.GetSize()) / 2.0))
    centering_transform.SetOffset(np.array(transform.GetInverse().TransformPoint(img_center) - reference_center))

    # centered_transform = sitk.Transform(transform)
    # centered_transform.AddTransform(centering_transform)

    centered_transform = sitk.CompositeTransform([transform, centering_transform])

    # Using the linear interpolator as these are intensity images, if there is a need to resample a ground truth
    # segmentation then the segmentation image should be resampled using the NearestNeighbor interpolator so that
    # no new labels are introduced.

    return sitk.Resample(img, reference_image, centered_transform, interpolator, 0.0)

In [13]:
folder = '../results/fbMOLLI_post/group/jointcorrelation/l2/image_loss_weight1/weight1/bspline/cps4_svfsteps7_svfscale1/e100'

original_folder = '/Users/mona/Documents/data/registration/voxelmorph/fbMOLLI_original_post'

output_folder = os.path.join(folder, 'moved_mat')
os.makedirs(output_folder, exist_ok=True)
registed_subjects = glob.glob(os.path.join(folder, 'moved/*.nii'))
for file in tqdm(registed_subjects):
    name = Path(file).stem
    print(f"The file name is {name}")
    original_name = os.path.join(original_folder, f"{name}.nii.gz")
    img = sitk.ReadImage(original_name)
    orig_shape = img.GetSize()
    print(orig_shape)
    
    nii2mat(file, os.path.join(output_folder, f"{name}.mat"))

  8%|██████████████▏                                                                                                                                                           | 1/12 [00:00<00:01,  9.35it/s]

The file name is 6263556_20131023_fbMOLLI_1
(240, 240, 20)
The file name is 0387056_20140416_fbMOLLI_3
(240, 240, 20)
The file name is 3386874_20140528_fbMOLLI_3
(240, 240, 20)
The file name is 7762959_20140319_fbMOLLI_1
(256, 256, 20)
The file name is 7762959_20140319_fbMOLLI_2


 75%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                          | 9/12 [00:00<00:00, 28.08it/s]

(256, 256, 20)
The file name is 8559809_20140402_fbMOLLI_3
(224, 224, 20)
The file name is 4790541_20131002_fbMOLLI_1
(320, 320, 20)
The file name is 3135857_20140528_fbMOLLI_2
(256, 256, 20)
The file name is 9034739_20140528_fbMOLLI_2
(256, 256, 20)
The file name is 8515597_20140611_fbMOLLI_2
(240, 240, 20)
The file name is 4930261_20140514_fbMOLLI_2
(320, 320, 20)
The file name is 8286778_20131113_fbMOLLI_2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 25.98it/s]

(288, 288, 20)


In [14]:
orig_shape[:-1]

(288, 288)